In [1]:
#imports
import os
import pandas as pd
import networkx as nx
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
import numpy as np
import json
from openpyxl import load_workbook
from unicodedata import normalize

In [2]:
dir = "planilhas/"

def get_file_name(filepath):
    """Lista os arquivos da pasta"""
    f = os.listdir(filepath)
    files = [os.path.splitext(filename)[0] for filename in os.listdir(dir)]
    return files

In [3]:
def get_sheet_names_xlsx(filepath):
    """Lista a sheet do arquivo"""
    wb = load_workbook(filepath, read_only=True, keep_links=False)
    return wb.sheetnames

In [4]:
def get_all_files():
    """Loop para listar todos os arquivos e suas sheets
def get_all_files():"""
    f = np.array(get_file_name(dir))
    sheets = []
    file_name = []
    for i in range(len(f)):
        file_name.append(f.item(i))
        s = get_sheet_names_xlsx(dir+file_name[i]+".xlsx")
        s_as_string = ' '.join([str(elem) for elem in s])   #converte a lista para string
        sheets.append(s_as_string)                          #nova lista
    print("Files: {}".format(file_name))
    print("Sheets: {}".format(sheets))
    return convert_all_to_json(dir, file_name, sheets, ".xlsx")

In [5]:
def convert_to_json(dir, file_name, sheet_name, ext):
    """Converte excel para Json"""
    excel_data_df = pd.read_excel(dir + file_name + ext, sheet_name=sheet_name)
    json_str = excel_data_df.to_json(orient='records')
    #print("The original dictionary is : {}".format(json_str))
    return handle_accentuation_json(json_str)

In [6]:
def handle_accentuation_json(json_str):
    """Tratamento de acentuação no Json"""
    json_str = json_str.encode("latin1").decode("unicode_escape")
    json_result = normalize('NFKD', json_str).encode('ASCII', 'ignore').decode('ASCII')
    #print(json_result)
    return json_result

In [7]:
def struct_json(json_str):
    """Estrutura do json"""
    parsed = json.loads(json_str)
    json_data = (json.dumps(parsed, indent=4, sort_keys=False))

    # printing Structed dictionary
    print("The structed dictionary is : {}".format(json_data))
    # return save_json(parsed)

In [8]:
def convert_all_to_json(dir, f, sheet_name, ext):
    """Loop para converter excel para Json"""
    list_json = []
    for i in range(len(f)):
        list_json.append({"switches": convert_to_json(dir, str(f[i]), sheet_name[i], ext)})
    save_json(list_json)


In [9]:
def convert_json_to_pandasDF(json_data):
    """Converte o Json em um dataframe"""
    dataset = pd.json_normalize(json_data)
    df = pd.read_json('output/data.json', orient='records' )
    # print(df.iloc[:3,:])

In [10]:
from networkx.readwrite import json_graph
def save_json(json_data):
    file_name = 'output/data.json'
    with open(file_name, 'w') as outfile:
        json.dump(json_data, outfile, indent=4)

    return convert_json_to_pandasDF(json_data)



In [11]:
#chamada funções
def main():
    get_all_files()

#main
if __name__ == '__main__':
    main()

Files: ['cmn01', 'core01', 'core02', 'icr01', 'pacs01']
Sheets: ['PORTAS_CORE', 'PORTAS_CORE', 'PORTAS_CORE', 'PORTAS_CORE', 'PORTAS_CORE']


<ipython-input-6-904a60c1ac22>:3: DeprecationWarning: invalid escape sequence '\/'
  json_str = json_str.encode("latin1").decode("unicode_escape")
